# 1 - Web Scraping / Scrapeo con BeatifulSoup (Bs4)
<p align = 'center'>
<img src = 'https://editor.analyticsvidhya.com/uploads/75676cover.jfif'/>
</p>

Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web de manera automatizada y masiva. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs.

En los últimos años el web scraping se ha convertido en una **técnica muy utilizada** dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro recurso a **falta de una API o un feed RSS**. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

Todo aquello que necesite interactividad con la página web va a necesitar **selenium**, una librería que nos permite darle instrucciones a nuestro explorador para poder simular los movimientos que haríamos por dentro de la web:
 - Cambiar de página
 - Rellenar formularios
 - Hacer click en botones
 - Cerrar pop-ups
 - Rechazar cookies
 
Por otro lado, todo aquello que sea estático podemos tirar únicamente de **BeautifulSoup**, la primera librería para scrapeo que vamos a conocer que nos permite, una vez cargada la página de la que vamos a extraer la información, recorrer el código HTML y quedarnos con aquellos datos que necesitemos.

Tendremos que realizar scrapeo cuando no dispongamos de una fuente más sencilla de la que obtener información como una API, una base de datos, fichero, etc...

# 2 - Extracción de los datos del HTML

Para scrapear necesitamos saber que pinta tiene la **estructura general** que tiene un HTML.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`, `<span>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$


Las etiquetas pueden tener atributos. Por ejemplo:

`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>`

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página, varios elementos pueden tener la misma clase.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca, no puede haber dos etiquetas con el mismo id.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las **etiquetas** que nos interesan dentro de todo el HTML de la página web.

Para ello vamos a usar **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [98]:
%pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [99]:
import requests as req


In [100]:

from bs4 import BeautifulSoup as bs #este alias es standard

# 3 - Scrapeando **WIKIPEDIA**

Vamos a echar un ojo a esta web **[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)** para intentar sacar la información de la tabla que contiene la esperanza de vida de los países europeos.

Hasta ahora habíamos visto `requests`para hacer llamadas a url (normalmente endpoints de las APIS que hemos estado mirando)

¿Y si llamamos a una url de una web digamos `normal`?

In [102]:
#url de la pagina a scrapear
url = 'https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

html = req.get(url).text #obtenemos el contenido de la pagina
html[:1000] #mostramos los primeros 1000 caracteres

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/List_of_European_countries_by_life_expectancy (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000013F04FACAD0>: Failed to resolve 'en.wikipedia.org' ([Errno 11001] getaddrinfo failed)"))

In [4]:
str(html).split('/head>')[1][:1000] #mostramos los primeros 1000 caracteres de la seccion body / Esto es una gualtrapada para ver el codigo html de la pagina

'\n<body class="skin--responsive skin-vector skin-vector-search-vue mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject mw-editable page-List_of_European_countries_by_life_expectancy rootpage-List_of_European_countries_by_life_expectancy skin-vector-2022 action-view"><a class="mw-jump-link" href="#bodyContent">Jump to content</a>\n<div class="vector-header-container">\n\t<header class="vector-header mw-header">\n\t\t<div class="vector-header-start">\n\t\t\t<nav class="vector-main-menu-landmark" aria-label="Site">\n\t\t\t\t\n<div id="vector-main-menu-dropdown" class="vector-dropdown vector-main-menu-dropdown vector-button-flush-left vector-button-flush-right"  >\n\t<input type="checkbox" id="vector-main-menu-dropdown-checkbox" role="button" aria-haspopup="true" data-event-name="ui.dropdown-vector-main-menu-dropdown" class="vector-dropdown-checkbox "  aria-label="Main menu"  >\n\t<label id="vector-main-menu-dropdown-label" for="vector-main-menu-dropdown-checkbox" class="vector-dr

Esto es infumable y para nada práctico. Para ello BeautifulSoup entra en juego para facilitarnos la vida.

Vimos en la docu, la manera de pasarle un `html` para que nos lo parsee (traduzca/convierta) en un objeto de BeautifulSoup muchos más manejable, amigable y con muchos métodos que nos facilitan el poder navegar el `html` para encontrar el contenido exacto que queremos extraer.

Así, al **parsearlo**, es mucho más **legible**. A este objeto por convención se le llama `soup`.

In [5]:
soup = bs(html, 'html.parser') #creamos el objeto soup, y le decimos que lo que le pasamos tiene pinta de html
type(soup)

bs4.BeautifulSoup

Vamos a acceder a la tabla que hay en la web. 

En primer lugar es necesario entender cómo se estructura una tabla en código HTML.

Gracias a BeautifulSoup podemos encontrar los diferentes **elementos** de una manera muy **veloz**.

Hay varias funciones para encontrar elementos concretos. `find()` y `find_all()` son dos ejemplos.

Vamos a ver como es la **estructura general** de una tabla en html:

![](https://www.corelangs.com/html/tables/img/html-table-structure.png)

![](https://www.ourtutorials.in/html/img/table1.JPG)

In [7]:
# Tabla
tabla = soup.find('table') #buscamos la tabla, en este caso solo hay una. Si hubiera mas, solo nos devolveria la primera. Si queremos todas, usamos find_all
# soup.find_all('table')  -->  En caso de que tuviéramos más de una tabla esta sería la línea de código correcta para acceder a todas ellas
tabla

<table class="wikitable sortable mw-datatable static-row-numbers sticky-header-multi sort-under col1left col21right" style="text-align:center;">
<tbody><tr>
<th rowspan="2" style="vertical-align:middle;">Countries and<br/>territories
</th>
<th colspan="7">Life expectancy for population in general
</th>
<th colspan="4" style="border-left-width:2px;">Life expectancy for male
</th>
<th colspan="4" style="border-left-width:2px;">Life expectancy for female
</th>
<th colspan="4" style="border-left-width:2px;"><style data-mw-deduplicate="TemplateStyles:r1038841319">.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}</style><span class="rt-commentedText tooltip tooltip-dotted" title="Difference in life expectancy for female and male">Sex gap</span>
</th>
<th rowspan="2" style="border-left-width:2px; text-align:middle;"><link href="mw-data:TemplateStyles:r1038841319" rel="mw-deduplicated-inline-style"/><span class="rt-commentedText tooltip tooltip-dotted" title="Total popula

In [37]:
tabla.prettify()

'<table class="box-More_citations_needed plainlinks metadata ambox ambox-content ambox-Refimprove" role="presentation">\n <tbody>\n  <tr>\n   <td class="mbox-image">\n    <div class="mbox-image-div">\n     <span typeof="mw:File">\n      <a class="mw-file-description" href="/wiki/File:Question_book-new.svg">\n       <img alt="" class="mw-file-element" data-file-height="399" data-file-width="512" decoding="async" height="39" src="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/50px-Question_book-new.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/75px-Question_book-new.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/100px-Question_book-new.svg.png 2x" width="50"/>\n      </a>\n     </span>\n    </div>\n   </td>\n   <td class="mbox-text">\n    <div class="mbox-text-span">\n     This article\n     <b>\n      needs additional citations for\n      <a href="/wiki/Wikipedia:Verifiability" title="Wikip

In [9]:
print(tabla.prettify()[:1000])

<table class="wikitable sortable mw-datatable static-row-numbers sticky-header-multi sort-under col1left col21right" style="text-align:center;">
 <tbody>
  <tr>
   <th rowspan="2" style="vertical-align:middle;">
    Countries and
    <br/>
    territories
   </th>
   <th colspan="7">
    Life expectancy for population in general
   </th>
   <th colspan="4" style="border-left-width:2px;">
    Life expectancy for male
   </th>
   <th colspan="4" style="border-left-width:2px;">
    Life expectancy for female
   </th>
   <th colspan="4" style="border-left-width:2px;">
    <style data-mw-deduplicate="TemplateStyles:r1038841319">
     .mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}
    </style>
    <span class="rt-commentedText tooltip tooltip-dotted" title="Difference in life expectancy for female and male">
     Sex gap
    </span>
   </th>
   <th rowspan="2" style="border-left-width:2px; text-align:middle;">
    <link href="mw-data:TemplateStyles:r1038841319" rel="

In [42]:
# Filas de la tabla
filas = tabla.find_all('tr') # tr --> table row
len(filas) #Acordaros que vimos que find all nos devolvía una lista con todos los elementos encontrados

1

In [43]:
type(filas) # ResultSet es una lista

bs4.element.ResultSet

Al ser una lista puedo recorrer cada elemento.

In [50]:
# Texto del primer elemento de la lista
filas[0].text

'This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources in this article. Unsourced material may be challenged and removed.Find sources:\xa0"List of countries by GNI (nominal) per capita"\xa0–\xa0news\xa0· newspapers\xa0· books\xa0· scholar\xa0· JSTOR (April 2024) (Learn how and when to remove this message)'

In [51]:
print(filas[0].text)

This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources in this article. Unsourced material may be challenged and removed.Find sources: "List of countries by GNI (nominal) per capita" – news · newspapers · books · scholar · JSTOR (April 2024) (Learn how and when to remove this message)


In [75]:
# texto de cada fila solo para los 3 primeros elementos
for f in filas[:3]:
    print(type(f.text), f.text)
    print('-----------------')

<class 'str'> This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources in this article. Unsourced material may be challenged and removed.Find sources: "List of countries by GNI (nominal) per capita" – news · newspapers · books · scholar · JSTOR (April 2024) (Learn how and when to remove this message)
-----------------


In [74]:
#vamos a sacar el texto de 3 primeros elementos del cuerpo de la tabla
for f in filas[2:5]:
    print(type(f.text), f.text)
    print('-----------------')

In [ ]:
f.text.split('\n')

In [ ]:
# Hagamos una lista, que contenga una lista por fila
# strip() --> quita los espacios que haya delante y detrás del texto
# split() --> va a separar la lista por la secuencia de caracteres '\n' para hacer una lista
filas_limpias = [f.text.strip().split('\n') for f in filas]
filas_limpias[:5]

In [ ]:
# La primera fila es la cabecera, y hay que limpiar los espacios en blanco
cabecera = filas_limpias[0]
cabecera

In [ ]:
# vamos a arreglarlo
final = [] # aquí el resultado final limpio
for fila in filas_limpias:
    tmp = [] # lista vacía temporal
    for elemento in fila:
        if elemento != '':
            tmp.append(elemento)
    final.append(tmp)

final[:5]

Hasta ahora hemos cogido la información contenida en una página que tiene una tabla. Hemos identificado la tabla y hemos accedido a ella. Con `find()` hemos buscado la etiqueta que nos interesa para ello (`<table>`). Una vez obtenido el HTML únicamente de la tabla lo hemos traducido para que sea algo más legible y hemos arreglado su contenido para poder llegar al siguiente paso: crear nuestro DataFrame.

In [ ]:
final[0] # primer elemento de la lista final

In [ ]:
final[1] # segundo elemento de la lista final

In [ ]:
final[2]

Para poder crear el DataFrame a partir de listas es necesario que todas ellas tengan la **misma cantidad de elementos**, tanto la que dará nombre a las columnas como la que contendrá la información de las filas.

In [ ]:
import pandas as pd

In [ ]:
nombres_columnas = final[1] # la primera fila la omitimos, nos quedamos la segunda, pero tenemos que arreglarla
nombres_columnas

In [12]:
data = final[2:] # desde la primera fila con información, la de Liechtenstein, hasta el final

NameError: name 'final' is not defined

In [ ]:
len(data[0]), len(nombres_columnas) # las filas tienen info para más columnas que la lista que dará nombre a las columnas...

En este caso no coinciden. Así que tendremos que hacer algún pequeño arreglo extra antes de conseguir hacer el DataFrame.

In [ ]:
nombres_columnas

In [ ]:
data[0]

In [ ]:
nombres_columnas.insert(0, 'Country') # al añadir en primer lugar 'Country' ya tenemos tantos elementos como en el resto, que serán las filas

In [ ]:
df = pd.DataFrame(data, columns = nombres_columnas)

In [ ]:
df.head(10)

In [5]:
import requests as req


In [ ]:
#RETO

#Vamos a escrapear la tabla de esta web --> https://en.wikipedia.org/wiki/List_of_countries_by_GNI_(nominal)_per_capita
#Escrapeamos la primera tabla, y cuando lo tengamos vamos con la segunda
#La salida debe ser un dataframe limpito,

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable



ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [24]:
import pandas as pd

In [26]:
from bs4 import BeautifulSoup as bs #este alias es standard

In [27]:
url= 'https://en.wikipedia.org/wiki/List_of_countries_by_GNI_(nominal)_per_capita'
html = req.get(url).text #obtenemos el contenido de la pagina
#html[:1000] #mostramos los primeros 1000 caracteres


In [28]:
soup = bs(html,'html.parser')
#type(soup)
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of countries by GNI (nominal) per capita - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-lim

In [29]:
# Busca la primera tabla, tiene la clase {'class': 'wikitable'} asi esta cuando uso el inspector
tabla = soup.find('table' , {'class': 'wikitable'})
tabla

<table class="wikitable sortable sticky-header sort-under col2left" style="text-align:right">
<tbody><tr>
<th>
</th>
<th>Country
</th>
<th>GNI per<br/>capita (US$)<sup class="reference" id="cite_ref-2"><a href="#cite_note-2"><span class="cite-bracket">[</span>2<span class="cite-bracket">]</span></a></sup>
</th>
<th>Year
</th></tr>
<tr>
<td>
</td>
<td><b>High Income Economies</b>
</td>
<td>
</td>
<td>2023
</td></tr>
<tr>
<td>–</td>
<td><i><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="500" data-file-width="1000" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Flag_of_Bermuda.svg/23px-Flag_of_Bermuda.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Flag_of_Bermuda.svg/35px-Flag_of_Bermuda.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Flag_of_Bermuda.svg/46px-Flag_of_Bermuda.svg.png 2x" width="23"/></span></span> </span><a href="/

In [32]:
filas = tabla.find_all('tr') # tr --> table row
len(filas) #Acordaros que vimos que find all nos devolvía una lista con todos los elementos encontrados


211

In [70]:
type(filas) # ResultSet es una lista

bs4.element.ResultSet

In [33]:
# al crear una lista puedo recorrerla
filas[0].text 

'\n\n\nCountry\n\nGNI percapita (US$)[2]\n\nYear\n'

In [40]:
filas_limpias = [f.text.split('\n') for f in filas]
filas_limpias = [[celdas.strip() for celdas in f] for f in filas_limpias]
filas_limpias

[['', '', '', 'Country', '', 'GNI percapita (US$)[2]', '', 'Year', ''],
 ['', '', '', 'High Income Economies', '', '', '', '2023', ''],
 ['', '–', 'Bermuda (UK)', '134,640', '2022', ''],
 ['', '1', 'Liechtenstein', '116,600', '2009', ''],
 ['', '2', 'Norway', '102,460', '2023', ''],
 ['', '3', 'Switzerland', '95,160', '2023', ''],
 ['', '4', 'Luxembourg', '88,370', '2023', ''],
 ['', '5', 'Ireland', '80,390', '2023', ''],
 ['', '6', 'United States', '80,300', '2023', ''],
 ['', '7', 'Iceland', '79,840', '2023', ''],
 ['', '–', 'Isle of Man (UK)', '76,300', '2021', ''],
 ['', '–', 'Faroe Islands (Denmark)', '74,420', '2022', ''],
 ['', '8', 'Denmark', '73,360', '2023', ''],
 ['', '9', 'Singapore', '70,590', '2023', ''],
 ['', '10', 'Qatar', '70,070', '2022', ''],
 ['', '–', 'Cayman Islands (UK)', '68,790', '2022', ''],
 ['', '11', 'Australia', '63,140', '2023', ''],
 ['', '12', 'Sweden', '61,650', '2023', ''],
 ['', '13', 'Netherlands', '60,670', '2023', ''],
 ['', '–', 'Hong Kong (Chin

In [41]:
# Aqui obtengo los datos de la tabla limpios de una vez

filas_final = []
for registro in filas_limpias:
    tmp = []
    for celda in registro:
        if celda != '':
            tmp.append(celda)
    filas_final.append(tmp)
      
     
   
     
print(f'{filas_final[:4 ]}')

[['Country', 'GNI percapita (US$)[2]', 'Year'], ['High Income Economies', '2023'], ['–', 'Bermuda (UK)', '134,640', '2022'], ['1', 'Liechtenstein', '116,600', '2009']]


In [42]:
for fila in filas_final:
    if len(fila) == 3:
        fila.insert(0, '')
    elif len(fila) == 2:
        fila.insert(0, '')
        fila.insert(0, '')
print(f'{filas_final[:4 ]}')

[['', 'Country', 'GNI percapita (US$)[2]', 'Year'], ['', '', 'High Income Economies', '2023'], ['–', 'Bermuda (UK)', '134,640', '2022'], ['1', 'Liechtenstein', '116,600', '2009']]


In [43]:
len(filas_final)

211

In [38]:
Data = [0]

In [44]:
df = pd.DataFrame(filas_final[1:], columns=filas_final[0] )
df
# falta limpiar la data 

,,Country,GNI percapita (US$)[2],Year
0,,,High Income Economies,2023
1,–,Bermuda (UK),"134,640",2022
2,1,Liechtenstein,"116,600",2009
3,2,Norway,"102,460",2023
4,3,Switzerland,"95,160",2023
...,...,...,...,...
205,187,Madagascar,530,2023
206,187,Mozambique,530,2023
207,189,Central African Republic,470,2023
208,190,Afghanistan,360,2022


# 4 - Geolocalizando una IP

**¿Dónde estoy?** --> https://tools.keycdn.com/geo

Esta web nos da toda la información de la IP que introducimos.

In [ ]:
url='https://tools.keycdn.com/geo'

In [ ]:
html = req.get(url).content

WTF? Que ha pasado



que es un [user agent] ??(https://www.zenrows.com/blog/user-agent-web-scraping#what-is)

User agents for web scraping --> [link](https://useragentstring.com/pages/Browserlist/)

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

response = req.get('http://httpbin.org/headers', headers=headers)

print(response.status_code)
print(response.text)

Se ha tenido que añadir un diccionario que contiene 'User-agent'. Hay muchos sitios web que van a poner impedimentos para que podamos obtener la información a través del scrapeo. Estamos haciendo una petición sin cabecera y no puede identificar el navegador que se está utilizando para acceder.

Cuando accedemos a una web pedimos acceso aportando mucha información en los metadatos: quién soy, dónde estoy...

Al hacer un `get` a pelo no está la información del navegador, se nos detecta como bot y se nos corta el acceso.

Esta información se contiene en lo que se llama **cabecera**. Tal como indico en el diccionario le estoy diciendo a esta web que estoy accediendo desde el navegador Mozilla en su versión 5.0. Estoy intentando sacar información de la web que, a priori, no debería poder sacar. Esta web pone un impedimento para ello, pero hay una solución, la cabecera (el **`header=`**).


In [ ]:
http://httpbin.org/headers

In [ ]:
https://useragentstring.com/pages/Browserlist/

In [ ]:
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

response = requests.get('http://httpbin.org/headers', headers=headers)

print(response.status_code)
print(response.text)


In [ ]:
# soup parseada para tener el HTML legible
soup = bs(html, 'html.parser')

In [ ]:
print(soup.prettify()[:1000]) # con .prettify() lo que hago es poner el código todavía más legible, con indentación
                       # .prettify() funciona distinto dentro que fuera de la función print()

Utilizando el inspector en el navegador podemos activar el cursor y encontrar en qué parte del código HTML se encuentra el elemento que necesitamos. Una vez identificado lo buscamos en nuestra *sopa*

In [ ]:
print(soup.find('div', {'id': 'geoResult'}).prettify()) # conseguido con el inspector

In [ ]:
tabla = soup.find('div', {'id': 'geoResult'}) # le llamo tabla, pues acabará siendo nuestra tabla
info = tabla.find('div', {'class': 'bg-light medium rounded p-3'}) # la información que tendrá la tabla está aquí

Ahora que ya sé dónde está la información que necesito y tengo una variable que la alberga puedo buscar más profundamente hasta poder extraer los datos que necesito.

Toda la info la tengo dentro de ese div veo que lo tengo en una lista con la class='row mb-0'

Pero cuidado porque hay dos clases iguales.

Accedamos a info para que nos dé todos los dd que tenga la clase col-8 text-monospace. Así me va a dar todos los que hay con la misma etiqueta y la misma clase.

Por comodidad de lectura del código esto lo guardamos en una variable que se llama detalles, al fin y al cabo son los detalles de la información que contendrá la tabla que estamos haciendo.

In [ ]:
detalles = info.find_all('dd', {'class': 'col-8 text-monospace'}) #Elemento y añado tb la clase

Hago una lista con el texto que hay en cada elemento de detalles. Le llamo conexión pues son los detalles de la info de esa conexión IP.

In [ ]:
conexion = [e.text for e in detalles]
conexion

Necesito hacer un diccionario. `<dd>` es la información de la conexión. `<dt>` tiene el nombre de cada una de las filas (es la columna de la izquierda). Por ello cojo esa información y lo uso como `keys`, mientras que lo que hemos hecho hasta ahora serán los `values`.

In [ ]:
keys = [e.text for e in tabla.find_all('dt')]
keys

In [ ]:
dicc = {k:v for k,v in zip(keys,conexion)} #me hago un diccionario con los datos de la conexion

In [ ]:
dicc

In [ ]:
import pandas as pd
ip = pd.DataFrame(dicc, index = [0]) #al ser valores escalares hay que proporcionar índice
                                     #al aportar índice se genera una única fila
ip.T

**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

In [ ]:
#Mi url ha cambiado
https://tools.keycdn.com/geo?host=88.30.60.7

In [ ]:
url='https://tools.keycdn.com/geo?host=137.255.90.7' #tengo parámetros en la direccion en funcion de la IP que busque...🤔

html = req.get(url, headers = user_agent).content

soup=bs(html, 'html.parser')

Es igual que antes, pues la estructura es la misma, primero saco todos los `<dd>`luego todos los `<dt>`

In [ ]:
tabla = soup.find('div', {'id': 'geoResult'})
tabla.find_all('dd', {'class': 'col-8 text-monospace'})

In [ ]:
tabla.find_all('dt')

In [ ]:
#'{:2.2f}'.format(12.5436363636363)  # formato en strings de numeros

Puedo hacer una función que haga de golpe todo lo que hemos hecho hasta ahora con una dirección IP

In [ ]:
def geo(ip):

    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = user_agent).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    conexion = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    return conexion

Vimos que una url también puede tener diferentes parámetros. ¿Qué pasa si tengo una lista de IP de la que quiero obtener la información y hacerme un dataframe?

In [ ]:
list_ip = ['137.255.90.7', '195.255.90.10', '177.255.21.7']

In [ ]:
for ip in list_ip:
    print(geo(ip))


In [ ]:
# Inténtalo tú (alumnos)

In [230]:
import requests as req
from bs4 import BeautifulSoup as bs # este alias es estandar
import pandas as pd


In [252]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
url='https://tools.keycdn.com/geo?host=195.255.90.10'

html = req.get(url, headers = headers).content
soup = bs(html, 'html.parser')
soup


<!DOCTYPE html>

<html lang="en" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="57ed0a7e3b29ae8f3a50b7d383902359f80ff602" name="version"/>
<title>IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools</title>
<meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." name="description"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@KeyCDN" name="twitter:site"/>
<meta content="@KeyCDN" name="twitter:creator"/>
<meta content="https://tools.keycdn.com/geo?host=195.255.90.10" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools" property="og:title"/>
<meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." property="og:descr

In [233]:
tabla = soup.find('div', {'id': 'geoResult'})
print(tabla)

<div class="mt-4" id="geoResult">
<div class="bg-light medium rounded p-3">
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mb-2">Location</p> <dl class="row mb-0">
<dt class="col-4">City</dt><dd class="col-8 text-monospace">Helsinki</dd><dt class="col-4">Region</dt><dd class="col-8 text-monospace">Uusimaa (18)</dd><dt class="col-4">Postal code</dt><dd class="col-8 text-monospace">00141</dd><dt class="col-4">Country</dt><dd class="col-8 text-monospace">Finland (FI)</dd><dt class="col-4">Continent</dt><dd class="col-8 text-monospace">Europe (EU)</dd><dt class="col-4">Coordinates</dt><dd class="col-8 text-monospace">60.1797 (lat) / 24.9344 (long)</dd><dt class="col-4">Time</dt><dd class="col-8 text-monospace">2024-09-18 00:51:07 (Europe/Helsinki)</dd> </dl>
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mt-4 mb-2">Network</p>
<dl class="row mb-0">
<dt class="col-4">IP address</dt><dd class="col-8 text-monospace">1

In [234]:
infor_fila= tabla.find_all('dd', {'class': 'col-8 text-monospace'})

In [235]:
infor_fila = [celda.text for celda in  infor_fila]
infor_fila

['Helsinki',
 'Uusimaa (18)',
 '00141',
 'Finland (FI)',
 'Europe (EU)',
 '60.1797 (lat) / 24.9344 (long)',
 '2024-09-18 00:51:07 (Europe/Helsinki)',
 '195.255.90.10',
 'ws10.lonne.fi',
 'Elisa Oyj',
 '719']

In [236]:
infor_colum = tabla.find_all('dt')

In [237]:
infor_colum = [col.text for col in infor_colum]
infor_colum

['City',
 'Region',
 'Postal code',
 'Country',
 'Continent',
 'Coordinates',
 'Time',
 'IP address',
 'Hostname',
 'Provider',
 'ASN']

In [238]:
data = {k:v for k, v in zip(infor_colum, infor_fila)}
data

{'City': 'Helsinki',
 'Region': 'Uusimaa (18)',
 'Postal code': '00141',
 'Country': 'Finland (FI)',
 'Continent': 'Europe (EU)',
 'Coordinates': '60.1797 (lat) / 24.9344 (long)',
 'Time': '2024-09-18 00:51:07 (Europe/Helsinki)',
 'IP address': '195.255.90.10',
 'Hostname': 'ws10.lonne.fi',
 'Provider': 'Elisa Oyj',
 'ASN': '719'}

In [163]:
df = pd.DataFrame(data, index=[0])
df

,City,Region,Postal code,Country,Continent,Coordinates,Time,IP address,Hostname,Provider,ASN
0,Helsinki,Uusimaa (18),00141,Finland (FI),Europe (EU),60.1797 (lat) / 24.9344 (long),2024-09-17 15:43:19 (Europe/Helsinki),195.255.90.10,ws10.lonne.fi,Elisa Oyj,719


In [273]:
list_ip = ['137.255.90.7', '195.255.90.10', '177.255.21.7']

In [276]:
def pagina_ip(list_ip):

    # Recorrido de lista de ip
    #permisos para usar la pagina
        
    # diccionario donde voy a guardar la iformacion para hacer luego el data frame 

        data = []

    # Tengo que hacer lo mismo a las tres, hago mi bucle que busca cada ip y la recorre en la lista 

        for ip in list_ip:
                    
        # busco la url con el parametro ip
            url = 'https://tools.keycdn.com/geo?host={ip}'
        # mi respuesta
            html = req.get(url , headers=headers).content 
            # hago mi sopa 
            soup = bs(html, 'html.parser') 
            

            tabla = soup.find('div', {'id': 'geoResult'})

            infor_fila1= [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]
            print(f'{infor_fila1}')
            
            infor_colum1 = [e.text for e in tabla.find_all('dt', {'class': 'col-4'})]

            inf_ip = dict(zip(infor_colum1,infor_fila1))
            # data['ip'].append(dicc_ip)
            data.append(inf_ip)

            df= pd.DataFrame(data, index =list_ip)
            return df
        

   
    




In [277]:
respuesta = pagina_ip(['137.255.90.7', '195.255.90.10', '177.255.21.7'])
respuesta

['Madrid', 'Madrid (M)', '28029', 'Spain (ES)', 'Europe (EU)', '40.4697 (lat) / -3.709 (long)', '2024-09-18 01:09:32 (Europe/Madrid)', '77.211.4.250', '77.211.4.250', 'Vodafone Spain', '12430']


,City,Region,Postal code,Country,Continent,Coordinates,Time,IP address,Hostname,Provider,ASN
137.255.90.7,Madrid,Madrid (M),28029,Spain (ES),Europe (EU),40.4697 (lat) / -3.709 (long),2024-09-18 01:09:32 (Europe/Madrid),77.211.4.250,77.211.4.250,Vodafone Spain,12430
195.255.90.10,Madrid,Madrid (M),28029,Spain (ES),Europe (EU),40.4697 (lat) / -3.709 (long),2024-09-18 01:09:32 (Europe/Madrid),77.211.4.250,77.211.4.250,Vodafone Spain,12430
177.255.21.7,Madrid,Madrid (M),28029,Spain (ES),Europe (EU),40.4697 (lat) / -3.709 (long),2024-09-18 01:09:32 (Europe/Madrid),77.211.4.250,77.211.4.250,Vodafone Spain,12430


In [240]:
df2 = pd.DataFrame(data)
df2

,0
0,137.255.90.7
1,195.255.90.10
2,177.255.21.7


In [246]:
headers = headers
#mi lista de ip
lista_ip = ['137.255.90.7', '195.255.90.10', '177.255.21.7']

# diccionario donde voy a guardar la iformacion para hacer luego el data frame 

data = []

# Tengo que hacer lo mismo a las tres, hago mi bucle que busca cada ip y la recorre en la lista 

for ip in lista_ip:
    dicc_ip = {}
    # busco la url con el parametro ip
    url = f'https://tools.keycdn.com/geo?ip={ip}'
    html = req.get(url , headers=headers).content # mi respuesta
    soup = bs(html, 'html.parser') # hago mi sopa 
    # llamo a la tabla
    tabla = soup.find_all('div', {'id': 'geoResult'})
            
    infor_fila = [celda.text for celda in  tabla.find_all('dd', {'class': 'col-8 text-monospace'})]
    print(f'{infor_fila}')
   # infor_colum1 = tabla.find('dt', {'class': 'col-4'})
    #dicc_ip = {k :v for k, v in zip( infor_colum1, infor_fila1,)}
    #data.append(dicc_ip)
    #print(f'{dicc_ip}')
    

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [216]:
df4 = pd.DataFrame(dicc_ip, index=[0])
df4

,ip
0,NaN


,[City],[Region],[Postal code],[Country],[Continent],[Coordinates],[Time],[IP address],[Hostname],[Provider],[ASN]
0,Helsinki,Uusimaa (18),00141,Finland (FI),Europe (EU),60.1797 (lat) / 24.9344 (long),2024-09-16 20:38:53 (Europe/Helsinki),195.255.90.10,ws10.lonne.fi,Elisa Oyj,719


### Ejemplo LinkedIn

In [176]:
#Vamos intentar scrapear Linkedin
URL='https://www.linkedin.com/jobs/search/' #Esta es la direccion de la pagina de busqueda de empleo de Linkedin general

A esta direccion le podemos poner parametros para afinar nuestra búsqueda.

URL = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Madrid&refresh=true'

`?keywords=data` nos dice que la palabra clave que queremos buscar es 'data'
    
`&location=Madrid` nos dice que queremos buscar en Madrid
    
`&refresh=true` nos dice que queremos que nos devuelva los resultados más recientes

Podemos filtrar más nuestra búsqueda en la web y veremos que van apareciendo más ***parametros*** en la URL.

`&f_TPR=r120960` nos dice que empleos de la ultima semana. El parámetro va en segundos así que 60 * 60 * 24 * número de días atrás que queremos buscar

`&start={i*25}` donde i sería el número de página


In [177]:
#Vamos a intentar scrapear esta URL
import requests as req
from bs4 import BeautifulSoup as bs
url = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Espa%C3%B1a&refresh=true%27&position=1&pageNum=0'
html = req.get(url).content
soup=bs(html, 'html.parser')

In [178]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="d_jobs_guest_search" name="pageKey"/>
  <!-- -->
  <meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
  <meta content="en_US" name="locale"/>
  <meta data-app-version="2.0.1974" data-browser-id="98a07036-6770-4954-82b4-d31322693c3e" data-call-tree-id="AAYiVo0saYqDBCELnZcDwg==" data-dfp-member-lix-treatment="control" data-disable-jsbeacon-pagekey-suffix="false" data-dna-member-lix-treatment="control" data-enable-page-view-heartbeat-tracking="" data-human-member-lix-treatment="control" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;RfIo4N0jSPen4Q7m6qZJkQ==" data-service-name="jobs-guest-frontend" id="config"/>
  <link href="https://www.linkedin.com/jobs/data-jobs" rel="canonical"/>
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon

In [179]:
# Intentad sacar los ingredientes de la recete de esta sopa y obtener para cada oferta de trabajo:

lista_ofertas = []
#Listado de ofertas
ofertas = soup.find_all('div', {'class':'base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card'})

for oferta in ofertas[:3]:
    dicc_oferta = {} #Diccionario donde almacenamos datos de 1 oferta
    #Titulo oferta
    titulo_oferta = oferta.find('span', {'class':'sr-only'}).text.strip()
    dicc_oferta['titulo_oferta'] = titulo_oferta

    # - Empresa
    empresa = oferta.find('a', {'class':'hidden-nested-link'}).text.strip()
    dicc_oferta['empresa'] = empresa
    
    # - Ubicacion
    ubicacion = oferta.find('span', {'class':'job-search-card__location'}).text.strip()
    dicc_oferta['ubicacion'] = ubicacion
    
    # - Link de la empresa
    link_empresa = oferta.find('a', {'class':'hidden-nested-link'}).attrs['href']
    dicc_oferta['link_empresa'] = link_empresa

    # - Link de la oferta
    link_oferta = oferta.find('a', {'class':'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'}).attrs['href']
    dicc_oferta['link_oferta'] = link_oferta

     # - Fecha de publicacion
    fecha_publicacion = oferta.find('time', {'job-search-card__listdate--new'}).text.strip()
    dicc_oferta['fecha_publicacion'] = fecha_publicacion

    #Appendeamos a nuestra lista de diccionarios de respuesta
    lista_ofertas.append(dicc_oferta)    

In [180]:
dicc_oferta

{'titulo_oferta': 'Data Entry Clerk ( Remote 100%)',
 'empresa': 'Opinion Focus Panel',
 'ubicacion': 'New York, United States',
 'link_empresa': 'https://www.linkedin.com/company/opinionfocuspanelllc?trk=public_jobs_jserp-result_job-search-card-subtitle',
 'link_oferta': 'https://www.linkedin.com/jobs/view/data-entry-clerk-remote-100%25-at-opinion-focus-panel-4026274571?position=3&pageNum=0&refId=45DZrPpttvV5ZtqF993fkg%3D%3D&trackingId=FMe9hd2XzHNQ6GlTr4AImA%3D%3D&trk=public_jobs_jserp-result_search-card',
 'fecha_publicacion': '8 hours ago'}

In [ ]:
# aqui estoy buscando el link de la oferta
oferta.find('a', {'class':'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'}).attrs['href']

In [ ]:
# aqui estoy buscando la fecha de la oferta , le digo que me de el texto y le hago el strip para que limpie el texto
ofertas[1].find('time', {'job-search-card__listdate--new'}).text.strip()

In [184]:
df_linkedin = pd.DataFrame(dicc_oferta, index= [0])
df_linkedin

,titulo_oferta,empresa,ubicacion,link_empresa,link_oferta,fecha_publicacion
0,Data Entry Clerk ( Remote 100%),Opinion Focus Panel,"New York, United States",https://www.linkedin.com/company/opinionfocusp...,https://www.linkedin.com/jobs/view/data-entry-...,8 hours ago


In [ ]:
#Os recomiendo ir echando una ojeada al html y la sopa poco a poco para ver como esta estructurado
#Lo primero que tendremos que buscar será el elemento que contiene todas las ofertas....🤔

#PISTA -- base-search-card__info
#De aqui en adelante vosotros solos, tened presente que querremos guardar toda esa info en un dataframe

In [ ]:
h3_titulos = soup.find_all('h3',{'class': 'base-search-card__title'})
titulos_ofertas = [h3.text.replace('\n','').strip() for h3 in h3_titulos]
titulos_ofertas

In [ ]:
titulos_ofertas[0].replace('\n','').strip()

In [ ]:
# - Titulo
# - Empresa
# - Ubicacion
# - Link de la empresa
# - Link de la oferta
# - Fecha de publicacion

In [ ]:
# SOLUCION para el primer elemento que aparece
import pandas as pd

data=[]
tabla = soup.find('div', {'class': 'base-search-card__info'})

titulo = tabla.find('h3', {'class': 'base-search-card__title'}).text.split('–')[0].strip() # - Titulo .sprit() para quitar espacios
empresa = tabla.find('h4', {'class': 'base-search-card__subtitle'}).text.strip()  # - Empresa
ubicacion = tabla.find('span', {'class': 'job-search-card__location'}).text.strip() # - Ubicacion
link_empresa = tabla.find('a', {'class': 'hidden-nested-link'})['href'].split('?')[0] # - Link de la empresa
link_oferta = soup.find('a', {'class': 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'})['href'] # - Link de la oferta
fecha_publicacion = tabla.find('time')['datetime'].strip()  # - Fecha de publicacion

print(titulo)
print(empresa)
print(ubicacion)
print(link_empresa)
print(link_oferta)
print(fecha_publicacion)

data.append({
    'titulo': titulo,
    'empresa': empresa,
    'ubicacion': ubicacion,
    'link_empresa': link_empresa,
    'link_oferta': link_oferta,
    'fecha': fecha_publicacion
})

display(pd.DataFrame(data))

### REDDIT - https://www.reddit.com/r/Python/

In [ ]:
url = 'https://www.reddit.com/r/Python/'
html = req.get(url).content
soup = bs(html, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
articulos = soup.find_all('shreddit-post')
print(articulos[0].prettify())

In [ ]:
len(articulos)

Ya sé que tengo el autor del post dentro de `<span class="whitespace-nowrap">` y quiero conseguir el nombre de los autores de los diferentes posts de reddit

soup.find_all('span', {'class':'whitespace-nowrap'}) me va a dar una lista

pues por cada elemento de la lista dame el texto y límpialo. (hemos visto que cada autor empieza por 'u/' y eso nos da igual).

In [ ]:
#Lista Autores
autores =[autor.text.replace('u/','') for autor in soup.find_all('span', {'class':'whitespace-nowrap'})]
autores[2]

In [ ]:
#Titulos:
titulos = []
listado = soup.find_all('a', {'font-bold'})

#este find_all me devuelve artículo y autor [0] no es lo que quiero, pero [1] sí es lo que quiero
#necesito quedarme los impares
for i in range(len(listado)):
  if i%2 != 0:
    titulos.append(listado[i].text.replace('\n','').strip())
titulos

In [ ]:
#Tipo articulo
tipos = [tipo.text.replace('\n','').strip() for tipo in soup.find_all('div',{'class': 'md'})]

Cuando tengo una etiqueta sea la que sea dentro tengo su clase, id, y más atributos. href, title, target...

El contenido de los atributos también es accesible. Es como coger información de lo que pone por fuera de la caja. Puedo localizar cualquier etiqueta :

In [ ]:
#Queremos coger la info de los atributos
soup.find_all('div', {'class': 'grow overflow-hidden'})[0]#.find('a').attrs#['href']

In [ ]:
soup.find_all('div', {'class': 'grow overflow-hidden'})[0].find('a')

In [ ]:
soup.find_all('div', {'class':'grow overflow-hidden'})[0].find('a').attrs

In [ ]:
soup.find_all('div', {'class':'grow overflow-hidden'})[0].find('a').attrs['href']

### REPASO FUNCIONES

In [ ]:
# repaso funciones

def suma(*args):
    return sum(args)

suma(2, 2, 3, 45, 67, 890)

In [ ]:
def saludar(nombre, lang='es', colega=True):
    s=''

    if colega:
        s='colega!!!'

    if lang=='es':
        print('Hola {} {}'.format(nombre, s))

    else:
        print('Hello {} buddy!!!'.format(nombre))

In [ ]:
def saludar_multiple(*lst, lang='es', colega=True):
    for e in lst:
        saludar(e, lang, colega)

In [ ]:
saludar('Pepe')

In [ ]:
saludar(['Pepe', 'en'])

In [ ]:
saludar(*['Pepe', 'en'])

In [ ]:
saludar_multiple('Ana', 'Pepe', 'Juan', 'Maria', lang='en')

In [ ]:
nombres=['Ana', 'Pepe', 'Juan', 'Maria']

config={'lang': 'es', 'colega': True}

In [ ]:
saludar_multiple(*nombres, **config)

In [ ]:
def function (*args, **kwargs):
    return